# **Data Preprocessing**

In [1]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense


In [2]:

train_data_dir = '/content/drive/MyDrive/ML Project Data Set/training_set'
test_data_dir = '/content/drive/MyDrive/ML Project Data Set/test_set'
categories = ['Agreeableness', 'Conscientiousness', 'Extraversion', 'Neuroticism', 'Openness']


In [3]:
def preprocess_image(image_path):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    image = cv2.resize(image, (128, 128))  # Resize to 128x128
    image = image / 255.0  # Normalize pixel values
    return image



In [4]:
def load_data(data_dir):
    images = []
    labels = []
    for idx, category in enumerate(categories):
        category_path = os.path.join(data_dir, category)
        for img_name in os.listdir(category_path):
            img_path = os.path.join(category_path, img_name)
            image = preprocess_image(img_path)
            images.append(image)
            labels.append(idx)
    return np.array(images), np.array(labels)



In [5]:

train_images, train_labels = load_data(train_data_dir)

test_images, test_labels = load_data(test_data_dir)



In [6]:

train_images = np.expand_dims(train_images, axis=-1)  # Add channel dimension
test_images = np.expand_dims(test_images, axis=-1)
train_labels = to_categorical(train_labels, num_classes=len(categories))  # One-hot encoding
test_labels = to_categorical(test_labels, num_classes=len(categories))


# **Model Trainig**

In [7]:

def create_model():
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 1)),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dense(len(categories), activation='softmax')  # Multi-class classification
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

model = create_model()



In [9]:

model.fit(train_images, train_labels, epochs=25, validation_data=(test_images, test_labels))


Epoch 1/25
6/6 [==============================] - 6s 1s/step - loss: 1.1554 - accuracy: 0.5311 - val_loss: 1.4304 - val_accuracy: 0.4091
Epoch 2/25
6/6 [==============================] - 5s 737ms/step - loss: 1.0000 - accuracy: 0.6102 - val_loss: 1.4303 - val_accuracy: 0.5000
Epoch 3/25
6/6 [==============================] - 4s 707ms/step - loss: 0.8998 - accuracy: 0.6102 - val_loss: 1.5612 - val_accuracy: 0.3182
Epoch 4/25
6/6 [==============================] - 6s 976ms/step - loss: 0.8245 - accuracy: 0.7119 - val_loss: 1.6873 - val_accuracy: 0.4545
Epoch 5/25
6/6 [==============================] - 5s 793ms/step - loss: 0.6234 - accuracy: 0.8362 - val_loss: 1.7733 - val_accuracy: 0.4545
Epoch 6/25
6/6 [==============================] - 4s 721ms/step - loss: 0.4397 - accuracy: 0.9040 - val_loss: 1.8618 - val_accuracy: 0.3636
Epoch 7/25
6/6 [==============================] - 5s 881ms/step - loss: 0.3441 - accuracy: 0.9322 - val_loss: 1.9573 - val_accuracy: 0.3636
Epoch 8/25
6/6 [=======

# **Model Evaluation**

In [10]:

loss, accuracy = model.evaluate(test_images, test_labels)
print(f'Test Accuracy: {accuracy * 100:.2f}%')


2/2 [==============================] - 0s 76ms/step - loss: 3.4985 - accuracy: 0.4318
Test Accuracy: 43.18%


# **Prediction**

In [11]:

def predict_personality(image_path):
    image = preprocess_image(image_path)
    image = np.expand_dims(image, axis=-1)  # Expand dims to match input shape
    image = np.expand_dims(image, axis=0)  # Add batch dimension
    prediction = model.predict(image)
    predicted_label = np.argmax(prediction, axis=1)[0]
    return categories[predicted_label]


new_image_path = '/content/drive/MyDrive/Sacrificial Lambs/160122124623-01-national-handwriting-day.jpg'
print(predict_personality(new_image_path))


1/1 [==============================] - 0s 381ms/step
Openness
